In [3]:
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [4]:
X,y = make_classification(n_samples=10000, n_features=10,n_informative=3)
     

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [10]:
dt = DecisionTreeClassifier(random_state=42)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)

print("Decision Tree accuracy",accuracy_score(y_test,y_pred))

Decision Tree accuracy 0.8905


## Bagging

In [8]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.5,
    bootstrap=True,
    random_state=42
)

In [9]:
bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.5,
                  n_estimators=500, random_state=42)

In [11]:
y_pred = bag.predict(X_test)
     

In [12]:
accuracy_score(y_test,y_pred)

0.919

In [13]:
bag.estimators_samples_[0].shape

(4000,)

In [14]:
bag.estimators_features_[0].shape

(10,)

## Bagging using SVM

In [16]:
bag = BaggingClassifier(
    estimator=SVC(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    random_state=42
)

In [17]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Bagging using SVM",accuracy_score(y_test,y_pred))
     

Bagging using SVM 0.8965


## Pasting

In [18]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=False,
    random_state=42,
    verbose = 1,
    n_jobs=-1
)

In [19]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Pasting classifier",accuracy_score(y_test,y_pred))

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:   12.9s remaining:  1.1min
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:   13.6s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.0s remaining:    0.6s


Pasting classifier 0.918


[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.4s finished


## Random Subspaces

In [21]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=1.0,
    bootstrap=False,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)
     

In [22]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Subspaces classifier",accuracy_score(y_test,y_pred))
     

Random Subspaces classifier 0.9095


In [23]:
bag.estimators_samples_[0].shape
     
(8000,)

bag.estimators_features_[0].shape

(5,)

## Random Patches

In [24]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    max_features=0.5,
    bootstrap_features=True,
    random_state=42
)
     

In [25]:
bag.fit(X_train,y_train)
y_pred = bag.predict(X_test)
print("Random Patches classifier",accuracy_score(y_test,y_pred))
     

Random Patches classifier 0.9105


## OOB Score

In [26]:
bag = BaggingClassifier(
    estimator=DecisionTreeClassifier(),
    n_estimators=500,
    max_samples=0.25,
    bootstrap=True,
    oob_score=True,
    random_state=42
)
     

In [27]:
bag.fit(X_train,y_train)

BaggingClassifier(estimator=DecisionTreeClassifier(), max_samples=0.25,
                  n_estimators=500, oob_score=True, random_state=42)

In [28]:
bag.oob_score_

0.924125

In [29]:
y_pred = bag.predict(X_test)
print("Accuracy",accuracy_score(y_test,y_pred))
     

Accuracy 0.9165


Bagging Tips
Bagging generally gives better results than Pasting
Good results come around the 25% to 50% row sampling mark
Random patches and subspaces should be used while dealing with high dimensional data
To find the correct hyperparameter values we can do GridSearchCV/RandomSearchCV

In [31]:
from sklearn.model_selection import GridSearchCV

In [32]:
parameters = {
    'n_estimators': [50,100,500], 
    'max_samples': [0.1,0.4,0.7,1.0],
    'bootstrap' : [True,False],
    'max_features' : [0.1,0.4,0.7,1.0]
    }

In [33]:
search = GridSearchCV(BaggingClassifier(), parameters, cv=5)
     

In [ ]:
search.fit(X_train,y_train)

In [ ]:
search.best_params_
search.best_score_